In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.lines import Line2D
%matplotlib inline
from tikzplotlib import save as tikz_save
from matplotlib import pyplot as pp
from matplotlib import style
from multiprocessing.pool import ThreadPool
from scipy.spatial.distance import cdist

In [ ]:
a_k = .5
lambda_ = 1
delta = 0
p_set = np.linspace(0, 1., 3)
R_set = np.linspace(0, 100., 3)
eps = 1e-4
data_num = 10000
iterMax = 50
worker_num = 10
dataset = np.load('mnist.npz')
M, D1, D2 = dataset['x_train'].shape
x_train = dataset['x_train'].reshape(M, D1 * D2).T
x_train = np.vstack([x_train, np.ones((1, M))])
x_train = (x_train / np.linalg.norm(x_train, axis = 0))[:,:data_num]
y_train = dataset['y_train'].reshape(M, 1).astype(int)[:data_num]
tmp, D1, D2 = dataset['x_test'].shape
x_test = dataset['x_test'].reshape(tmp, D1 * D2).T
x_test = np.vstack([x_test, np.ones((1, tmp))])
x_test = (x_test / np.linalg.norm(x_test, axis = 0))[:,:data_num]
y_test = dataset['y_test'].reshape(tmp, 1).astype(int)[:data_num]

In [ ]:
workers_x = np.split(x_train, (np.arange(1, worker_num) * .1 * x_train.shape[1]).astype(int), axis = 1)
workers_y = np.split(np.array(y_train), (np.arange(1, worker_num) * .1 * x_train.shape[1]).astype(int), axis = 0)
def get_Index(y):
    Index = np.ones((y.shape[0], worker_num))
    Index[np.arange(0, y.shape[0]), y.reshape(y.shape[0])] = 0
    Index_not = np.ones(Index.shape) - Index
    return Index, Index_not

def SVM_classify(x, w):
    return np.argmax(np.matmul(x.T, w), axis = 1).reshape(x.shape[1], 1)

def SVM_loss(x, y, w, Index):
    return np.multiply(Index, (np.matmul(x.T, w) - np.repeat(np.multiply(x.T, w[:,y.reshape(y.shape[0])].T).sum(axis = 1).reshape(y.shape[0], 1), worker_num, axis = 1) + delta).clip(min = 0))

def reg_loss(x, y, w, lambda_, Index):
    return SVM_loss(x, y, w, Index).sum() + lambda_/2 * np.linalg.norm(w)**2

def SVM_loss_grad(x, y, w, lambda_, Index, Index_not):
    svm_loss = SVM_loss(x, y, w, Index) > 0
    x_rp = np.repeat(x.T, worker_num, axis = 0).reshape(x.shape[1], worker_num*x.shape[0])
    return (np.multiply(np.repeat(svm_loss - np.multiply(np.repeat(svm_loss.sum(axis=1).reshape(svm_loss.shape[0], 1), worker_num, axis=1), Index_not), x.shape[0], axis=1), x_rp).sum(axis=0).reshape(worker_num, x.shape[0]).T)

In [ ]:
def gd_back_track(w_k, a_k, grad_w, x, y, lambda_, Index):
    a = a_k
    w_k_1 = w_k - a * grad_w
    return w_k_1
def gd(x, y, lambda_, a_k):
    Index, Index_not = get_Index(y_train)
    w_k = np.random.rand((x.shape[0] * worker_num)).reshape(x.shape[0],worker_num)
    k = 0
    L1 = reg_loss(x, y, w_k, lambda_, Index)
    while True:
        L = L1
        k += 1
        grad_w = SVM_loss_grad(x, y, w_k, lambda_, Index, Index_not)
        w_k = gd_back_track(w_k, a_k, grad_w, x, y, lambda_, Index)
        L1 = reg_loss(x, y, w_k, lambda_, Index)
        print(k, L1, np.linalg.norm(grad_w))
        if np.abs(L-L1) < eps or k == iterMax:
            break
    return w_k.reshape(x.shape[0],worker_num)

In [ ]:
def dgd_worker(x, y, w_k, lambda_, Index, Index_not, p, R):
    grad_w = SVM_loss_grad(x, y, w_k, lambda_, Index, Index_not)
    noise_mask = np.random.rand(1) < p
    return np.array(grad_w) + noise_mask * np.random.normal(0, R, grad_w.size).reshape(grad_w.shape)

def dgd_master(w_k, a_k, grad_w, x, y, lambda_, Index):
    w_k = gd_back_track(w_k, a_k, grad_w.mean(axis = 0).reshape(w_k.shape), x, y, lambda_, Index)
    return w_k

In [ ]:
def dgd_worker_(args):
    return dgd_worker(*args)
#algorithm 1
def dgd(x, y, workers_x, workers_y, lambda_, a_k, p, R, master, worker_):
    Index = list(range(worker_num))
    Index_not = list(range(worker_num))
    for i in range(worker_num):
        Index[i], Index_not[i] = get_Index(workers_y[i])
    Index_all, Index_not_ll = get_Index(y)
    w_k = np.random.rand(x.shape[0], worker_num)
    k = 0
    cost = []
    L1 = reg_loss(x, y, w_k, lambda_, Index_all)
    while True:
        print(k)
        L = L1
        k += 1
        workers_w_k = np.array(worker_num * [w_k]).reshape(worker_num, x.shape[0], worker_num)
        zipped = zip(workers_x, workers_y, workers_w_k, worker_num * [lambda_], Index, Index_not, worker_num * [p], worker_num * [R])
        pool = ThreadPool(worker_num)
        results = pool.map(worker_, zipped)
        pool.close()
        pool.join()
        grad_w = np.array(results)
        w_k = master(w_k, a_k, grad_w, x, y, lambda_, Index_all)
        L1 = reg_loss(x, y, w_k, lambda_, Index_all)
        cost.append((k, L1))
        if np.abs(L-L1) < eps or k == iterMax:
            break
    print(k, L1, np.linalg.norm(grad_w))
    return w_k.reshape(x.shape[0],worker_num), k, cost


In [ ]:
def Iter_num_vs_p_and_R(algorithm, p_set, R_set, **arg_k_w):
    d = []
    for R in R_set:
        for p in p_set:
            arg_k_w['p'] = p
            arg_k_w['R'] = R
            print(p, R)
            w_k, k, cost = algorithm(**arg_k_w)
            d.append((w_k, k, cost, p, R))
    return d
arg_k_w = {'x':x_train, 'y':y_train, 'workers_x':workers_x, 'workers_y':workers_y, 'lambda_':lambda_, 'a_k':a_k, 'master':dgd_master, 'worker_':dgd_worker_}
tmp_p_set = Iter_num_vs_p_and_R(dgd, p_set, [R_set[1]], **arg_k_w)
tmp_R_set = Iter_num_vs_p_and_R(dgd, [p_set[1]], R_set, **arg_k_w)

In [ ]:
colors = iter(cm.rainbow(np.linspace(0, 1, len(tmp_p_set))))
legends = []
for i in range(len(tmp_p_set)):
    w_k, k, cost, p, R = tmp_p_set[i]
    k, loss = zip(*cost)
    plt.plot(k, loss, color=next(colors))
    plt.xlabel("Number of Iteration")
    plt.ylabel("Loss")
    legends.append('Dis. GD p = %.1f, R = %d' %(p,R))
plt.legend(legends)
tikz_save('Fig1.tex');
plt.show()
colors = iter(cm.rainbow(np.linspace(0, 1, len(tmp_R_set))))
legends = []
for i in range(len(tmp_R_set)):
    w_k, k, cost, p, R = tmp_R_set[i]
    k, loss = zip(*cost)
    plt.plot(k, loss, color=next(colors))
    legends.append('Dis. GD p = %.1f, R = %d' %(p,R))
    plt.xlabel("Number of Iteration")
    plt.ylabel("Loss")
plt.legend(legends)
tikz_save('Fig2.tex');
plt.show()

In [ ]:
#algorithm 1 robusted version
def robust_dgd_master(w_k, a_k, grad_w, x, y, lambda_, Index):
    grad_w = grad_w.reshape(worker_num, w_k.size)
    worker_weights = cdist(grad_w, grad_w,'cosine').sum(axis = 0).reshape(worker_num, 1)
    worker_weights[np.isnan(worker_weights)] = 1.
    worker_weights += 1
    worker_weights /= np.linalg.norm(worker_weights)
    w_k = gd_back_track(w_k, a_k, np.multiply(np.array(worker_weights).reshape(worker_num, 1), grad_w).reshape(worker_num, w_k.shape[0], worker_num).sum(axis = 0).reshape(w_k.shape), x, y, lambda_, Index)
    return w_k

#algorithm 1 robusted version
#Noise is averaged
def robust_dgd_worker(x, y, w_k, lambda_, Index, Index_not, p, R):
    grad_w = SVM_loss_grad(x, y, w_k, lambda_, Index, Index_not)
    noise = np.zeros(grad_w.shape)
    for i in range(worker_num):
        noise_mask = np.random.rand(1) < p
        noise += noise_mask * np.random.normal(0, R, grad_w.size).reshape(grad_w.shape)
    return np.array(grad_w) + noise / worker_num

def robust_dgd_worker_(args):
    return robust_dgd_worker(*args)

In [ ]:
arg_k_w = {'x':x_train, 'y':y_train, 'workers_x':workers_x, 'workers_y':workers_y, 'lambda_':lambda_, 'a_k':a_k, 'master':dgd_master, 'worker_':robust_dgd_worker_}
tmp_r_p_set = Iter_num_vs_p_and_R(dgd, p_set, [R_set[1]], **arg_k_w)
tmp_r_R_set = Iter_num_vs_p_and_R(dgd, [p_set[1]], R_set, **arg_k_w)

In [ ]:
colors = iter(cm.rainbow(np.linspace(0, 1, len(tmp_r_p_set))))
legends = []
for i in range(len(tmp_r_p_set)):
    w_k, k, cost, p, R = tmp_r_p_set[i]
    k, loss = zip(*cost)
    plt.plot(k, loss, color=next(colors))
    plt.xlabel("Number of Iteration")
    plt.ylabel("Loss")
    legends.append('Robust Dis. GD p = %.1f, R = %d' %(p,R))
plt.legend(legends)
tikz_save('Fig3.tex');
plt.show()

colors = iter(cm.rainbow(np.linspace(0, 1, len(tmp_r_R_set))))
legends = []
for i in range(len(tmp_r_R_set)):
    w_k, k, cost, p, R = tmp_r_R_set[i]
    k, loss = zip(*cost)
    plt.plot(k, loss, color=next(colors))
    legends.append('Robust Dis. GD p = %.1f, R = %d' %(p,R))
    plt.xlabel("Number of Iteration")
    plt.ylabel("Loss")
plt.legend(legends)
tikz_save('Fig4.tex');
plt.show()

In [ ]:
A = np.array([[1, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1]])
A = np.repeat(np.repeat(A.reshape(worker_num, worker_num, 1, 1), x_train.shape[0], axis = 2), worker_num, axis=3)

a_k = 1e-7
R_set = np.linspace(0, 0.01, 2)
iterMax = 50

def dsgm_agent(Ai, x, y, w_k, lambda_, a_k, Index, Index_not, p, R):
    bar_w_k = np.multiply(Ai, w_k).mean(axis = 0)
    grad_w = SVM_loss_grad(x, y, bar_w_k, lambda_, Index, Index_not)
    bar_w_k = gd_back_track(bar_w_k, a_k, grad_w, x, y, lambda_, Index)
    noise_mask = np.random.rand(1) < p
    return bar_w_k + noise_mask * np.random.normal(0, R, bar_w_k.size).reshape(bar_w_k.shape), grad_w

def dsgm_agent_(args):
    return dsgm_agent(*args)


def dsgm(x, y, workers_x, workers_y, lambda_, a_k, A, p, R, agent):
    Index = list(range(worker_num))
    Index_not = list(range(worker_num))
    for i in range(worker_num):
        Index[i], Index_not[i] = get_Index(workers_y[i])
    Index_all, Index_not_ll = get_Index(y)
    w_k = np.random.rand(worker_num, x.shape[0], worker_num)
    k = 0
    cost = []
    L1 = reg_loss(x, y, w_k.mean(axis = 0), lambda_, Index_all)
    while True:
        L = L1
        k += 1
        workers_w_k = np.array(worker_num * [w_k]).reshape(worker_num, worker_num, x.shape[0], worker_num)
        zipped = zip(A, workers_x, workers_y, workers_w_k, worker_num * [lambda_], worker_num * [a_k], Index, Index_not, worker_num * [p], worker_num * [R])
        pool = ThreadPool(worker_num)
        results = pool.map(dsgm_agent_, zipped)
        pool.close()
        pool.join()
        print(k)
        
        w_k, grad_w = zip(*results)
        w_k = np.array(w_k)
        grad_w = np.array(grad_w)
        L1 = reg_loss(x, y, w_k.mean(axis = 0), lambda_, Index_all)
        L1s = []
        for i in range(worker_num):
            L1s.append(reg_loss(x, y, w_k[i], lambda_, Index_all))
        cost.append((k, L1s))
        if np.abs(L-L1) < eps or k == iterMax:
            break
    print(k, L1, np.linalg.norm(grad_w.mean(axis = 0)))
    return w_k.mean(axis = 0).reshape(x.shape[0],worker_num), k, cost


In [ ]:
arg_k_w = {'x':x_train, 'y':y_train, 'workers_x':workers_x, 'workers_y':workers_y, 'lambda_':lambda_, 'a_k':a_k, 'A':A, 'agent':dsgm_agent}
tmp_dp_set = Iter_num_vs_p_and_R(dsgm, p_set, [R_set[1]], **arg_k_w)
tmp_dR_set = Iter_num_vs_p_and_R(dsgm, [p_set[1]], R_set, **arg_k_w)

In [ ]:
for i in range(len(tmp_dp_set)):
    legends = []
    w_k, k, cost, p, R = tmp_dp_set[i]
    k, losses = zip(*cost)
    colors = iter(cm.rainbow(np.linspace(0, 1, worker_num)))
    for l, j in enumerate(zip(*losses)):
        color = next(colors)
        plt.plot(k, j, color=color)    
        legends.append('Agent %d' %(l + 1))
    plt.xlabel("Number of Iteration Distributed SM p = %.1f, R = %.2f" %(p,R))
    plt.ylabel("Agents' Loss")
    plt.legend(legends)
    tikz_save('Fig5{:03d}.tex'.format(i));
    plt.show()

for i in range(len(tmp_dR_set)):
    legends = []
    w_k, k, cost, p, R = tmp_dR_set[i]
    k, losses = zip(*cost)
    colors = iter(cm.rainbow(np.linspace(0, 1, worker_num)))
    for l, j in enumerate(zip(*losses)):
        color = next(colors)
        plt.plot(k, j, color=color)    
        legends.append('Agent %d' %(l + 1))
    plt.xlabel("Number of Iteration Distributed SM p = %.1f, R = %.2f" %(p,R))
    plt.ylabel("Agents' Loss")
    plt.legend(legends)
    tikz_save('Fig6{:03d}.tex'.format(i));
    plt.show()

In [ ]:
def robust_dsgm_agent(Ai, x, y, w_k, lambda_, a_k, Index, Index_not, p, R):
    bar_w_k = np.multiply(Ai, w_k).mean(axis = 0)
    grad_w = SVM_loss_grad(x, y, bar_w_k, lambda_, Index, Index_not)
    bar_w_k = gd_back_track(bar_w_k, a_k, grad_w, x, y, lambda_, Index)
    noise = np.zeros(bar_w_k.shape)
    for i in range(50):
        noise_mask = np.random.rand(1) < p
        noise += noise_mask * np.random.normal(0, R, bar_w_k.size).reshape(bar_w_k.shape)
    return bar_w_k + noise / 50, grad_w
def robust_dsgm_agent_(args):
    return dsgm_agent(*args)
arg_k_w = {'x':x_train, 'y':y_train, 'workers_x':workers_x, 'workers_y':workers_y, 'lambda_':lambda_, 'a_k':a_k, 'A':A, 'agent':robust_dsgm_agent}
tmp_r_dp_set = Iter_num_vs_p_and_R(dsgm, p_set, [R_set[1]], **arg_k_w)
tmp_r_dR_set = Iter_num_vs_p_and_R(dsgm, [p_set[1]], R_set, **arg_k_w)

In [ ]:
for i in range(len(tmp_r_dp_set)):
    legends = []
    w_k, k, cost, p, R = tmp_r_dp_set[i]
    k, losses = zip(*cost)
    colors = iter(cm.rainbow(np.linspace(0, 1, worker_num)))
    for l, j in enumerate(zip(*losses)):
        color = next(colors)
        plt.plot(k, j, color=color)    
        legends.append('Agent %d' %(l + 1))
    plt.xlabel("Number of Iteration Robust Distributed SM p = %.1f, R = %.2f" %(p,R))
    plt.ylabel("Agents' Loss")
    plt.legend(legends)
    tikz_save('Fig7{:03d}.tex'.format(i));
    plt.show()
    

for i in range(len(tmp_r_dR_set)):
    legends = []
    w_k, k, cost, p, R = tmp_r_dR_set[i]
    k, losses = zip(*cost)
    colors = iter(cm.rainbow(np.linspace(0, 1, worker_num)))
    for l, j in enumerate(zip(*losses)):
        color = next(colors)
        plt.plot(k, j, color=color)    
        legends.append('Agent %d' %(l + 1))
    plt.xlabel("Number of Iteration Robust Distributed SM p = %.1f, R = %.2f" %(p,R))
    plt.ylabel("Agents' Loss")
    plt.legend(legends)
    tikz_save('Fig8{:03d}.tex'.format(i));
    plt.show()
